In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV

In [2]:
bcancer_df=pd.read_csv('BreastCancer.csv',index_col=0)

In [3]:
lbl=LabelEncoder()

In [4]:
bcancer_df['Class']=lbl.fit_transform(bcancer_df['Class'])
bcancer_df

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,0
63375,9,1,2,6,4,10,7,7,2,1
76389,10,4,7,2,2,8,6,1,1,1
95719,6,10,10,10,8,10,7,10,7,1
128059,1,1,1,1,2,5,5,1,1,0
...,...,...,...,...,...,...,...,...,...,...
1369821,10,10,10,10,5,10,10,10,7,1
1371026,5,10,10,10,4,10,5,6,3,1
1371920,5,1,1,1,2,1,3,2,1,0


In [5]:
X=bcancer_df.drop('Class',axis=1)
y=bcancer_df['Class']

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=23,stratify=y)

In [7]:
svm=SVC(probability=True,random_state=23)

In [8]:
lr=LogisticRegression()

In [9]:
nb=GaussianNB()

In [10]:
voting=VotingClassifier(estimators=[('SVM',svm),('LR',lr),('NB',nb)],voting="soft")

In [11]:
voting.fit(X_train,y_train)

VotingClassifier(estimators=[('SVM', SVC(probability=True, random_state=23)),
                             ('LR', LogisticRegression()),
                             ('NB', GaussianNB())],
                 voting='soft')

In [12]:
y_pred_prob=voting.predict_proba(X_test)[:,1]

In [13]:
log_loss(y_test,y_pred_prob)

0.1377657178736647

In [14]:
y_pred=voting.predict(X_test)

In [15]:
accuracy_score(y_test,y_pred)

0.9571428571428572

# Grid Search cv

In [16]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

In [17]:
voting.get_params()

{'estimators': [('SVM', SVC(probability=True, random_state=23)),
  ('LR', LogisticRegression()),
  ('NB', GaussianNB())],
 'flatten_transform': True,
 'n_jobs': None,
 'verbose': False,
 'voting': 'soft',
 'weights': None,
 'SVM': SVC(probability=True, random_state=23),
 'LR': LogisticRegression(),
 'NB': GaussianNB(),
 'SVM__C': 1.0,
 'SVM__break_ties': False,
 'SVM__cache_size': 200,
 'SVM__class_weight': None,
 'SVM__coef0': 0.0,
 'SVM__decision_function_shape': 'ovr',
 'SVM__degree': 3,
 'SVM__gamma': 'scale',
 'SVM__kernel': 'rbf',
 'SVM__max_iter': -1,
 'SVM__probability': True,
 'SVM__random_state': 23,
 'SVM__shrinking': True,
 'SVM__tol': 0.001,
 'SVM__verbose': False,
 'LR__C': 1.0,
 'LR__class_weight': None,
 'LR__dual': False,
 'LR__fit_intercept': True,
 'LR__intercept_scaling': 1,
 'LR__l1_ratio': None,
 'LR__max_iter': 100,
 'LR__multi_class': 'auto',
 'LR__n_jobs': None,
 'LR__penalty': 'l2',
 'LR__random_state': None,
 'LR__solver': 'lbfgs',
 'LR__tol': 0.0001,
 'LR__v

In [18]:
params={'SVM__gamma': ['scale','auto'],'SVM__C':np.linspace(0.001,5,5),'LR__penalty':['l1','l2','elasticnet',None],'NB__var_smoothing':np.linspace(0.0001,0.999,5)}

In [19]:
gcv=GridSearchCV(voting,param_grid=params,verbose=3,cv=kfold,scoring='neg_log_loss')

In [20]:
gcv.fit(X,y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=scale;, score=nan total time=   0.1s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=scale;, score=nan total time=   0.1s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=scale;, score=nan total time=   0.1s
[CV 4/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=scale;, score=nan total time=   0.1s
[CV 5/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=scale;, score=nan total time=   0.1s
[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.0001, SVM__C=0.001, SVM__gamma=auto;, score=nan total time=   0.1s
[CV

[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 4/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 5/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.249825, SVM__C=2.500499999999

[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 4/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 5/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 4/5] END LR__penalty=l1, NB__var_smoothing=0.49955, SVM__C=5.0, SVM__ga

[CV 5/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=0.001, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 3/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 4/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 5/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=nan total time=   0.0s
[CV 1/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=auto;, score=nan total time=   0.1s
[CV 2/5] END LR__penalty=l1, NB__var_smoothing=0.999, SVM__C=1.2507499999999998, SVM__gamma=auto;, score=nan tot

[CV 2/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.071 total time=   0.1s
[CV 3/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.055 total time=   0.1s
[CV 4/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.104 total time=   0.1s
[CV 5/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.146 total time=   0.1s
[CV 1/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=3.75025, SVM__gamma=scale;, score=-0.117 total time=   0.1s
[CV 2/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=3.75025, SVM__gamma=scale;, score=-0.076 total time=   0.1s
[CV 3/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=3.75025, SVM__gamma=scale;, score=-0.056 total time=   0.0s
[CV 4/5] END LR__penalty=l2, NB__var_smoothing=0.0001, SVM__C=3.75025, SVM__gamma=scale;, score=-0.096 tot

...

[CV 2/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=0.001, SVM__gamma=auto;, score=-0.061 total time=   0.1s
[CV 3/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=0.001, SVM__gamma=auto;, score=-0.072 total time=   0.1s
[CV 4/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=0.001, SVM__gamma=auto;, score=-0.111 total time=   0.1s
[CV 5/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=0.001, SVM__gamma=auto;, score=-0.165 total time=   0.1s
[CV 1/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=-0.101 total time=   0.1s
[CV 2/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=-0.076 total time=   0.1s
[CV 3/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=1.2507499999999998, SVM__gamma=scale;, score=-0.055 total time=   0.1s
[CV 4/5] END LR__penalty=None, NB__var_smoothing=0.249825, SVM__C=1.2507499999999998, SVM__gamma=

[CV 4/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=-0.087 total time=   0.1s
[CV 5/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=scale;, score=-0.188 total time=   0.0s
[CV 1/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.088 total time=   0.1s
[CV 2/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.077 total time=   0.1s
[CV 3/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.052 total time=   0.1s
[CV 4/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.092 total time=   0.1s
[CV 5/5] END LR__penalty=None, NB__var_smoothing=0.49955, SVM__C=2.5004999999999997, SVM__gamma=auto;, score=-0.166 total time=   0.1s
[CV 1/5] END LR__penalty=None, NB__var_smoothing=0.49

[CV 3/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=3.75025, SVM__gamma=auto;, score=-0.055 total time=   0.1s
[CV 4/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=3.75025, SVM__gamma=auto;, score=-0.092 total time=   0.1s
[CV 5/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=3.75025, SVM__gamma=auto;, score=-0.161 total time=   0.1s
[CV 1/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=5.0, SVM__gamma=scale;, score=-0.091 total time=   0.1s
[CV 2/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=5.0, SVM__gamma=scale;, score=-0.085 total time=   0.1s
[CV 3/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=5.0, SVM__gamma=scale;, score=-0.056 total time=   0.1s
[CV 4/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=5.0, SVM__gamma=scale;, score=-0.088 total time=   0.1s
[CV 5/5] END LR__penalty=None, NB__var_smoothing=0.749275, SVM__C=5.0, SVM__gamma=scale;, score=-0.176 total time=   0.1s
[CV 1/5] END LR

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
500 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
250 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_voting.py", line 349, 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=VotingClassifier(estimators=[('SVM',
                                                     SVC(probability=True,
                                                         random_state=23)),
                                                    ('LR',
                                                     LogisticRegression()),
                                                    ('NB', GaussianNB())],
                                        voting='soft'),
             param_grid={'LR__penalty': ['l1', 'l2', 'elasticnet', None],
                         'NB__var_smoothing': array([1.00000e-04, 2.49825e-01, 4.99550e-01, 7.49275e-01, 9.99000e-01]),
                         'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'SVM__gamma': ['scale', 'auto']},
             scoring='neg_log_loss', verbose=3)

In [21]:
gcv.best_params_

{'LR__penalty': None,
 'NB__var_smoothing': 0.999,
 'SVM__C': 0.001,
 'SVM__gamma': 'scale'}

In [22]:
gcv.best_score_

-0.09459790624917777